# Imports

In [3]:
# Packages
import pandas as pd

# Helper functions
from ipynb.fs.full.functions import *

# Other notebooks for combining code
from ipynb.fs.full.semrush_call import execute_call
from ipynb.fs.full.text_division_and_styling import LoopGetHeaderCountFromURL, LoopGetListsCountFromURL
from ipynb.fs.full.semrush_beoordeling_concurrentieniveau import getKeywordDifficulty 
from ipynb.fs.full.InboundOutboundScraper import loop_get_all_website_links
from ipynb.fs.full.SemRush_InboundOutbound_Backlink import get_all_website_links_semrush

# Executing of functions
## Variables for keyword search

In [12]:
keyword = 'bouwmaterialen' # What keyword do you want to call for?
result_count = '5' # How many keyword results do you want to gather data for?

# Do you want to overwrite the SEMrush keyword call if it has already been executed before?
# This is to save on SEMrush token usage
potential_overwrite = False

# Do you want to save the result of the end dataframe?
save = True

## Creating dataframes

In [13]:
def ExecuteAllCalls():
    # Define variables for the SEMrush call and gather keyword data
    export_columns = 'Dn,Ur'
    endpoint_url = 'https://api.semrush.com/'
    main_dataframe = execute_call(keyword, result_count, export_columns, endpoint_url, potential_overwrite)
    
    # Add used keyword to the dataframe
    keyword_list = []
    keyword_list += int(result_count) * [keyword]
    main_dataframe['Keyword'] = keyword_list
    
    # Executing functions from other notebooks to gather data
    print("  0% - Executing getKeywordDifficulty")
    KeywordDifficultyFunction = getKeywordDifficulty(keyword, potential_overwrite)
    KeywordDifficultyList = []
    KeywordDifficultyList += int(result_count) * [KeywordDifficultyFunction['Kd'][0]]
    main_dataframe['Kd'] = KeywordDifficultyList
    
    print(" 25% - Executing LoopGetHeaderCountFromURL")
    main_dataframe['HeaderCount'] = LoopGetHeaderCountFromURL(main_dataframe)
    print(" 50% - Executing LoopGetListsCountFromURL")
    main_dataframe['ListsCount'] = LoopGetListsCountFromURL(main_dataframe)
    print(" 75% - Executing loop_get_all_website_links")
    website_links = loop_get_all_website_links(main_dataframe)
    main_dataframe['InternalLinksSoup'] = website_links[0]
    main_dataframe['ExternalLinksSoup'] = website_links[1]
    print(" 90% - Executing get_all_website_links_semrush")
    main_dataframe['InternalLinksSEMrush'] = get_all_website_links_semrush(main_dataframe, 'internal', potential_overwrite)
    main_dataframe['ExternalLinksSEMrush'] = get_all_website_links_semrush(main_dataframe, 'external', potential_overwrite)
    main_dataframe['BacklinksSEMrush'] = get_all_website_links_semrush(main_dataframe, 'backlink', potential_overwrite)
    
    # Set correct index, potentially save the output and return the dataframe
    main_dataframe = main_dataframe.set_index('Number')
    main_dataframe_hash = hash_variable(main_dataframe)
    if save == True:
        main_dataframe.to_csv(f"calls\dataframe-{main_dataframe_hash}.csv")
    print("100% - Done")
    return main_dataframe

In [14]:
main_dataframe = ExecuteAllCalls()
main_dataframe

Deze call is al een keer uitgevoerd, we gaan verder met de bestaande resultaten.
Dit bestand is laatst gewijzigd op: Thu Apr 29 11:22:11 2021
  0% - Executing getKeywordDifficulty
Deze call is al een keer uitgevoerd, we gaan verder met de bestaande resultaten.
Dit bestand is laatst gewijzigd op: Thu Apr 29 16:07:55 2021
 25% - Executing LoopGetHeaderCountFromURL
 50% - Executing LoopGetListsCountFromURL
 75% - Executing loop_get_all_website_links
 90% - Executing get_all_website_links_semrush
100% - Done


,Dn,Ur,Keyword,Kd,HeaderCount,ListsCount,InternalLinksSoup,ExternalLinksSoup,InternalLinksSEMrush,ExternalLinksSEMrush,BacklinksSEMrush
Number,,,,,,,,,,,
0,bouwmaat.nl,https://www.bouwmaat.nl/bouwmaterialen,bouwmaterialen,84.13,4,31,1425,11,43,77,6884
1,hornbach.nl,https://www.hornbach.nl/shop/Bouwmateriaal/S44...,bouwmaterialen,84.13,10,17,217,7,42,79,633
2,bouwbestel.nl,https://www.bouwbestel.nl/bouwmaterialen.html,bouwmaterialen,84.13,2,7,2595,14,128,19,12
3,online-bouwmaterialen.nl,https://www.online-bouwmaterialen.nl/,bouwmaterialen,84.13,3,31,138,10,2,2,36
4,bouwonline.com,https://www.bouwonline.com/,bouwmaterialen,84.13,7,7,1029,3,42,79,7
